# Liver lobule analysis
### For Natalie Porat-Shliom lab

### 12/07/23
### Author: Andy D. Tran, CCR Microscopy Core, LCBG, CCR, NCI

### Use Cellpose models for cell, mitochondria, and lipid droplet segmentation. Generate EDT maps for
### user-defined central and portal veins. Output intensity, geometric and spatial parameters.

### ----------------------------------------------------------------------------
### Navigation

[Segmentation and quantification](#main_function)
    
[Load existing segmentation](#alt_function)

[Quantification](#quantification)


In [1]:
# Import libraries--------------------------------------------------------------------------------
import os 
import numpy as np 
import cv2
import pandas as pd 
import napari 
import re

from scipy import ndimage as nd 
from tifffile import imread, imwrite
from skimage.measure import regionprops
from tqdm import tqdm
from cellpose import models, io

In [2]:
# Define functions---------------------------------------------------------------------------------
def model_apply(model, img):
    mask, flows, style = model.eval(img, channels = [0, 0])
    
    return mask

def object_quant(lbl_img, lipid_img):
    df = pd.DataFrame() 
    
    roi_id = [] 
    lipid_int = [] 
    
    roiprops = regionprops(lbl_img, intensity_image = lipid_img)
    
    for roi in tqdm(range(len(roiprops))):
        roi_id.append(roiprops[roi].label)
        lipid_int.append(roiprops[roi].mean_intensity)
        
    df['roi_id'] = roi_id
    df['lipid_int'] = lipid_int
    
    return df

def object_filter(lbl_img, int_df, int_thresh):
    roi_id = lbl_img.reshape(-1)
    
    mask_df = pd.DataFrame()
    mask_df['roi_id'] = roi_id
    mask_df = pd.merge(mask_df, int_df, how = 'left', on = ['roi_id'])
    
    mask_tmp = mask_df['lipid_int'].to_numpy()
    mask_tmp = mask_tmp.reshape(lbl_img.shape)
    
    mask = np.where(mask_tmp <= int_thresh, 0, lbl_img)
    
    return mask

def roi_quant(lbl_img, cell_img, mito_img, lipid_img, cv_edt, pv_edt, organelle_edt):
    df = pd.DataFrame() 
    
    roi_id = [] 
    cell_id = [] 
    area = [] 
    perimeter = []
    eccentricity = [] 
    solidity = [] 
    axis_major_length = []
    axis_minor_length = [] 
    feret_diameter_max = [] 
    centroid_x = [] 
    centroid_y = [] 
    mito_int = [] 
    lipid_int = [] 
    cv_min_dist = [] 
    cv_max_dist = [] 
    cv_mean_dist = [] 
    pv_min_dist = [] 
    pv_max_dist = [] 
    pv_mean_dist = []
    organelle_min_dist = []
    organelle_max_dist = [] 
    organelle_mean_dist = [] 
    
    cellprops = regionprops(lbl_img, intensity_image = cell_img)
    mitoprops = regionprops(lbl_img, intensity_image = mito_img)
    lipidprops = regionprops(lbl_img, intensity_image = lipid_img)
    cvprops = regionprops(lbl_img, intensity_image = cv_edt)
    pvprops = regionprops(lbl_img, intensity_image = pv_edt)
    organelleprops = regionprops(lbl_img, intensity_image = organelle_edt)
    
    for roi in tqdm(range(len(cellprops))):
        roi_id.append(cellprops[roi].label)
        cell_id.append(round(cellprops[roi].mean_intensity))
        area.append(cellprops[roi].area)
        perimeter.append(cellprops[roi].perimeter)
        eccentricity.append(cellprops[roi].eccentricity)
        solidity.append(cellprops[roi].solidity)
        axis_major_length.append(cellprops[roi].axis_major_length)
        axis_minor_length.append(cellprops[roi].axis_minor_length)
        feret_diameter_max.append(cellprops[roi].feret_diameter_max)
        centroid_x.append(cellprops[roi].centroid[1])
        centroid_y.append(cellprops[roi].centroid[0])
        mito_int.append(mitoprops[roi].mean_intensity)
        lipid_int.append(lipidprops[roi].mean_intensity)
        cv_min_dist.append(cvprops[roi].min_intensity)
        cv_max_dist.append(cvprops[roi].max_intensity)
        cv_mean_dist.append(cvprops[roi].mean_intensity)
        pv_min_dist.append(pvprops[roi].min_intensity)
        pv_max_dist.append(pvprops[roi].max_intensity)
        pv_mean_dist.append(pvprops[roi].mean_intensity)
        organelle_min_dist.append(organelleprops[roi].min_intensity)
        organelle_max_dist.append(organelleprops[roi].max_intensity)
        organelle_mean_dist.append(organelleprops[roi].mean_intensity)
        
    df['roi_id'] = roi_id
    df['cell_id'] = cell_id
    df['area'] = area
    df['perimeter'] = perimeter
    df['eccentricity'] = eccentricity
    df['solidity'] = solidity
    df['axis_major_length'] = axis_major_length
    df['axis_minor_length'] = axis_minor_length
    df['feret_diameter_max'] = feret_diameter_max
    df['centroid_x'] = centroid_x
    df['centroid_y'] = centroid_y
    df['mito_int'] = mito_int
    df['lipid_int'] = lipid_int
    df['cv_min_dist'] = cv_min_dist
    df['cv_max_dist'] = cv_max_dist
    df['cv_mean_dist'] = cv_mean_dist
    df['pv_min_dist'] = pv_min_dist
    df['pv_max_dist'] = pv_max_dist
    df['pv_mean_dist'] = pv_mean_dist
    df['organelle_min_dist'] = organelle_min_dist
    df['organelle_max_dist'] = organelle_max_dist
    df['organelle_mean_dist'] = organelle_mean_dist
    
    return df 

### Segmentation and quantification mode

<a id='main_function'></a>

### Set paths

<a id='set_path'></a>

In [4]:
# Define model paths--------------------------------------------------------------------------------------

cell_model_path = input('Cell model path: ')
mito_model_path = input('Mito model path: ')
lipid_model_path = input('Lipid model path: ')

Cell model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/cell/models/brownl_cell_01
Mito model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/mito/models/brownl_mito_03
Lipid model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/lipid/models/brownl_lipid_01


In [201]:
# Define file paths--------------------------------------------------------------------------

input_path = input('Input path: ')
output_path = input('Output path: ')

Input path:  Z:\Analysis\[NCI] [TGIMB] Natalie Porat-Shliom\Lauryn Brown\images\PLIN5 variants in ND\PLIN5E (1-424)
Output path:  Z:\Analysis\[NCI] [TGIMB] Natalie Porat-Shliom\Lauryn Brown\output\PLIN5 variants in ND\PLIN5E (1-424)


In [202]:
input_path = os.path.normpath(input_path)
output_path = os.path.normpath(output_path)

cell_model_path = os.path.normpath(cell_model_path)
mito_model_path = os.path.normpath(mito_model_path)
lipid_model_path = os.path.normpath(lipid_model_path)

cell_model = models.CellposeModel(gpu = True, pretrained_model = cell_model_path)
mito_model = models.CellposeModel(gpu = True, pretrained_model = mito_model_path)
lipid_model = models.CellposeModel(gpu = True, pretrained_model = lipid_model_path)

### Load image

<a id='load_cell'></a>

In [738]:
# Load image-----------------------------------------------------------------------------------------------
tmp_list = os.listdir(input_path)

img_list = [] 
for tmp in tmp_list:
    if re.search('.tif', tmp):
        img_list.append(tmp)
        
print(img_list)
print(len(img_list))
img_index = input('Select image: ')
img_index = int(img_index) - 1
print(img_list[img_index])

['3907C M1_L1_Cropped PP-PC Axis 1.tif', '3907C M1_L2_Cropped PP-PC Axis 2.tif', '3907C M1_L2_Cropped PP-PC Axis 3.tif', '3907C M1_L2_Cropped PP-PC Axis 4.tif', '3907C M1_L3_Cropped PP-PC Axis 5.tif', '3907C M1_L3_Cropped PP-PC Axis 6.tif', '3920E M2_L1_Cropped PP-Pc Axis 1.tif', '3920E M2_L1_Cropped PP-PC Axis 2.tif', '3920E M2_L1_Cropped PP-PC Axis 3.tif', '3920E M2_L1_Cropped PP-PC Axis 4.tif', '3920E M2_L2_Cropped PP-PC Axis 5.tif', '3920E M2_L2_Cropped PP-PC Axis 6.tif']
12


Select image:  12


3920E M2_L2_Cropped PP-PC Axis 6.tif


In [739]:
img_path = os.path.join(input_path, img_list[img_index])
img = imread(img_path)

print(img.shape)

viewer = napari.view_image(img)

(3, 1440, 5184)


In [740]:
actin_ch_number = int(input('Input actin channel: ')) - 1
mito_ch_number = int(input('Input mitochondria channel: ')) - 1
lipid_ch_number = int(input('Input lipid channel: ')) - 1

viewer.close()

Input actin channel:  3
Input mitochondria channel:  1
Input lipid channel:  2


In [741]:
actin_img = img[actin_ch_number, :, :]
mito_img = img[mito_ch_number, :, :]
lipid_img = img[lipid_ch_number, :, :]

### Segmentation--------------------------------------------------------

###### Cell segmentation

In [742]:
mito_tmp = nd.gaussian_filter(mito_img, sigma = 50)
mito_tmp = mito_tmp * 3

tmp_img = np.add(actin_img, mito_tmp)

cell_mask = model_apply(cell_model, tmp_img)
cell_mask = np.uint16(cell_mask)

In [743]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
cell_layer = viewer.add_labels(cell_mask)

In [744]:
cell_mask = viewer.layers['cell_mask'].data
cell_mask = np.uint16(cell_mask)

In [745]:
cell_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cell.tif'))
imwrite(cell_mask_path, cell_mask)

viewer.close()

In [746]:
#cell_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cell.tif'))
#
#cell_mask = imread(cell_mask_path)

###### Mitochondria segmentation

In [747]:
mito_mask = model_apply(mito_model, mito_img)
mito_mask = np.uint16(mito_mask)

In [748]:
viewer = napari.view_image(mito_img, colormap = 'green', blending = 'additive')
mito_layer = viewer.add_labels(mito_mask)

In [749]:
mito_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito.tif'))
imwrite(mito_mask_path, mito_mask)

viewer.close()

In [750]:
#mito_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito.tif'))
#
#mito_mask = imread(mito_mask_path)

###### Lipid segmentation

In [ ]:
lipid_mask_tmp = model_apply(lipid_model, lipid_img)
lipid_mask_tmp = np.uint16(lipid_mask_tmp)

lipid_df_tmp = object_quant(lipid_mask_tmp, lipid_img)
lipid_mask = object_filter(lipid_mask_tmp, lipid_df_tmp, 10)

In [704]:
viewer = napari.view_image(lipid_img, colormap = 'magenta', blending = 'additive')
lipid_layer = viewer.add_labels(lipid_mask)

In [705]:
lipid_mask = viewer.layers['lipid_mask'].data

In [706]:
lipid_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid.tif'))
imwrite(lipid_mask_path, lipid_mask)

viewer.close()

In [707]:
#lipid_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid.tif'))

#lipid_mask = imread(lipid_mask_path)

### Organelle overlap----------------------------------------

In [708]:
mito_binary = np.where(mito_mask > 0, 1, 0)
lipid_binary = np.where(lipid_mask > 0 , 1, 0)

overlap_tmp = np.add(mito_binary, lipid_binary)
overlap_mask = np.where(overlap_tmp == 2, cell_mask, 0)

In [709]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')
overlap_layer = viewer.add_labels(overlap_mask)
cell_layer = viewer.add_labels(cell_mask)

In [710]:
overlap_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_overlap.tif'))
imwrite(overlap_mask_path, overlap_mask)

viewer.close()

In [711]:
#overlap_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_overlap.tif'))
#
#overlap_mask = imread(overlap_mask_path)

### EDT maps--------------------------------------------------

###### Central vein EDT map

In [712]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')

In [713]:
cv_mask = viewer.layers['Labels'].data
cv_mask = np.where(cv_mask == 1, 0, 1)
cv_edt = nd.distance_transform_edt(cv_mask)

In [714]:
cv_edt_layer = viewer.add_image(cv_edt, contrast_limits = [0, 65535])

In [715]:
cv_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cv_edt.tif'))
imwrite(cv_edt_path, cv_edt)

viewer.close()

###### Portal vein EDT map

In [716]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')

In [717]:
pv_mask = viewer.layers['Labels'].data
pv_mask = np.where(pv_mask == 1, 0, 1)
pv_edt = nd.distance_transform_edt(pv_mask)

In [718]:
pv_edt_layer = viewer.add_image(pv_edt, contrast_limits = [0, 65535])

In [719]:
pv_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_pv_edt.tif'))
imwrite(pv_edt_path, pv_edt)

viewer.close()

###### Mitochondria EDT map

In [720]:
mito_binary = np.where(mito_mask == 0, 1, 0)
mito_edt = nd.distance_transform_edt(mito_binary)

In [721]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')
mito_edt_layer = viewer.add_image(mito_edt)

In [722]:
mito_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito_edt.tif'))
imwrite(mito_edt_path, mito_edt)

viewer.close()

###### Lipid EDT map

In [723]:
lipid_binary = np.where(lipid_mask == 0, 1, 0)
lipid_edt = nd.distance_transform_edt(lipid_binary)

In [724]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')
lipid_edt_layer = viewer.add_image(lipid_edt)

In [725]:
lipid_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid_edt.tif'))
imwrite(lipid_edt_path, lipid_edt)

viewer.close()

### Quantification----------------------------------------------------

<a id='quantification'></a>

###### Portal to central vein distance

In [970]:
pv_df_tmp = roi_quant(pv_mask, cell_mask, mito_img, lipid_img, cv_edt, pv_edt, mito_edt)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.28it/s]


In [971]:
pv_cv_dist = pv_df_tmp['cv_max_dist'][0]

print(pv_cv_dist)

4207.83424103184


###### Cell statistics

In [972]:
cell_df_tmp = roi_quant(cell_mask, cell_mask, mito_img, lipid_img, cv_edt, pv_edt, mito_edt)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:03<00:00, 28.24it/s]


In [973]:
cell_df = pd.DataFrame()
cell_df['cell_id'] = cell_df_tmp['roi_id']
cell_df['cell_area'] = cell_df_tmp['area']
cell_df['cell_centroid_x'] = cell_df_tmp['centroid_x']
cell_df['cell_centroid_y'] = cell_df_tmp['centroid_y']
cell_df['cell_mito_int'] = cell_df_tmp['mito_int']
cell_df['cell_lipid_int'] = cell_df_tmp['lipid_int']
cell_df['cell_cv_min_dist'] = cell_df_tmp['cv_min_dist']
cell_df['cell_cv_max_dist'] = cell_df_tmp['cv_max_dist']
cell_df['cell_cv_mean_dist'] = cell_df_tmp['cv_mean_dist']
cell_df['cell_pv_min_dist'] = cell_df_tmp['pv_min_dist']
cell_df['cell_pv_max_dist'] = cell_df_tmp['pv_max_dist']
cell_df['cell_pv_mean_dist'] = cell_df_tmp['pv_mean_dist']

print(cell_df.head())
print(cell_df.shape)

   cell_id  cell_area  cell_centroid_x  cell_centroid_y  cell_mito_int  \
0        1    11590.0       862.311907        36.183089      54.404918   
1        2    23499.0      1046.306056        55.725776      55.780076   
2        3    16461.0      1272.940951        34.444566      68.364802   
3        4    69358.0      1546.880490       161.415136      58.434384   
4        5    39972.0      1725.781822        78.483213      57.589663   

   cell_lipid_int  cell_cv_min_dist  cell_cv_max_dist  cell_cv_mean_dist  \
0        3.951682       3232.634375       3415.578575        3317.178111   
1        3.872761       3020.309421       3249.609977        3132.342625   
2        4.595347       2789.393124       3040.481047        2908.040019   
3        4.690649       2464.569942       2797.390570        2627.604679   
4        3.310142       2311.493240       2614.688700        2453.549567   

   cell_pv_min_dist  cell_pv_max_dist  cell_pv_mean_dist  
0        393.288698        522.280576  

###### Mitochondria statistics

In [ ]:
mito_df_tmp = roi_quant(mito_mask, cell_mask, mito_img, lipid_img, cv_edt, pv_edt, lipid_edt)

 93%|███████████████████████████████████████████████████████████████████████▍     | 7648/8244 [00:16<00:01, 469.13it/s]

In [948]:
mito_df = mito_df_tmp.merge(cell_df, how = 'outer', on = 'cell_id')
mito_df['organelle'] = 'mito'
print(mito_df.head())
print(mito_df.shape)

   roi_id  cell_id   area  perimeter  eccentricity  solidity  \
0       1        0  267.0  60.284271      0.602512  0.956989   
1       8        0   95.0  34.384776      0.610150  0.950000   
2       9        0  128.0  40.384776      0.658208  0.962406   
3      10        0  100.0  38.520815      0.805022  0.909091   
4      31        0  136.0  44.384776      0.806179  0.931507   

   axis_major_length  axis_minor_length  feret_diameter_max   centroid_x  ...  \
0          20.772306          16.578585           21.633308   185.089888  ...   
1          12.423153           9.842691           13.341664   658.221053  ...   
2          14.856726          11.184678           15.524175   694.062500  ...   
3          14.909523           8.845006           15.524175   954.570000  ...   
4          17.192779          10.172477           18.027756  2981.330882  ...   

   cell_centroid_y  cell_mito_int  cell_lipid_int  cell_cv_min_dist  \
0              NaN            NaN             NaN        

###### Lipid statistics

In [949]:
lipid_df_tmp = roi_quant(lipid_mask, cell_mask, mito_img, lipid_img, cv_edt, pv_edt, mito_edt)

100%|█████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 390.59it/s]


In [950]:
lipid_df = lipid_df_tmp.merge(cell_df, how = 'outer', on = 'cell_id')
lipid_df['organelle'] = 'lipid'
print(lipid_df.head())
print(lipid_df.shape)

   roi_id  cell_id    area   perimeter  eccentricity  solidity  \
0    21.0       10   562.0   87.497475      0.281889  0.967298   
1    34.0        0   534.0   84.426407      0.389699  0.965642   
2   154.0        0  1620.0  148.124892      0.425077  0.978852   
3   737.0        0   800.0  103.497475      0.365061  0.973236   
4   744.0        0   769.0  101.497475      0.455531  0.973418   

   axis_major_length  axis_minor_length  feret_diameter_max   centroid_x  ...  \
0          27.360527          26.250979           28.861739  3356.679715  ...   
1          27.214209          25.062721           28.301943  5106.911985  ...   
2          47.764190          43.234098           50.328918  2912.625309  ...   
3          33.098807          30.814440           34.438351  3982.925000  ...   
4          33.166135          29.525146           34.438351  4048.932380  ...   

   cell_centroid_y  cell_mito_int  cell_lipid_int  cell_cv_min_dist  \
0          42.6985      33.672048        5.79

###### Overlap statistics

In [951]:
overlap_df_tmp = roi_quant(overlap_mask, cell_mask, mito_img, lipid_img, cv_edt, pv_edt, mito_edt)

100%|█████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 309.19it/s]


In [952]:
overlap_df = pd.DataFrame() 
overlap_df['cell_id'] = overlap_df_tmp['roi_id']
overlap_df['overlap_area'] = overlap_df_tmp['area']

print(overlap_df.head())
print(overlap_df.shape)

   cell_id  overlap_area
0       10          40.0
1       14          88.0
2       34           3.0
3       35          66.0
4       36           8.0
(17, 2)


###### Combine dataframes

In [953]:
df = pd.concat([mito_df, lipid_df])
df = df.merge(overlap_df, how = 'outer', on = 'cell_id')
df['pv_cv_dist'] = pv_cv_dist

print(df.head())
print(df.shape)

   roi_id  cell_id   area  perimeter  eccentricity  solidity  \
0     1.0        0  267.0  60.284271      0.602512  0.956989   
1     8.0        0   95.0  34.384776      0.610150  0.950000   
2     9.0        0  128.0  40.384776      0.658208  0.962406   
3    10.0        0  100.0  38.520815      0.805022  0.909091   
4    31.0        0  136.0  44.384776      0.806179  0.931507   

   axis_major_length  axis_minor_length  feret_diameter_max   centroid_x  ...  \
0          20.772306          16.578585           21.633308   185.089888  ...   
1          12.423153           9.842691           13.341664   658.221053  ...   
2          14.856726          11.184678           15.524175   694.062500  ...   
3          14.909523           8.845006           15.524175   954.570000  ...   
4          17.192779          10.172477           18.027756  2981.330882  ...   

   cell_lipid_int  cell_cv_min_dist  cell_cv_max_dist  cell_cv_mean_dist  \
0             NaN               NaN               Na

In [954]:
df_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_output.csv'))
df.to_csv(df_path, header = True)

[Load another image](#load_cell) [Set new path](#set_path) [New mito seg](#mito_path) [New lipid seg](#lipid_path)

### Test---------------------------------

In [75]:
df_tmp = pd.DataFrame()
df_tmp['relative_cell_cv_dist'] = df['cell_cv_max_dist'] / df['pv_cv_dist']
    
conditions = [
    df_tmp['relative_cell_cv_dist'].gt(11/12),
    df_tmp['relative_cell_cv_dist'].gt(10/12) & df_tmp['relative_cell_cv_dist'].le(11/12),
    df_tmp['relative_cell_cv_dist'].gt(9/12) & df_tmp['relative_cell_cv_dist'].le(10/12),
    df_tmp['relative_cell_cv_dist'].gt(8/12) & df_tmp['relative_cell_cv_dist'].le(9/12),
    df_tmp['relative_cell_cv_dist'].gt(7/12) & df_tmp['relative_cell_cv_dist'].le(8/12),
    df_tmp['relative_cell_cv_dist'].gt(6/12) & df_tmp['relative_cell_cv_dist'].le(7/12),
    df_tmp['relative_cell_cv_dist'].gt(5/12) & df_tmp['relative_cell_cv_dist'].le(6/12),
    df_tmp['relative_cell_cv_dist'].gt(4/12) & df_tmp['relative_cell_cv_dist'].le(5/12),
    df_tmp['relative_cell_cv_dist'].gt(3/12) & df_tmp['relative_cell_cv_dist'].le(4/12),
    df_tmp['relative_cell_cv_dist'].gt(2/12) & df_tmp['relative_cell_cv_dist'].le(3/12),
    df_tmp['relative_cell_cv_dist'].gt(1/12) & df_tmp['relative_cell_cv_dist'].le(2/12),
    df_tmp['relative_cell_cv_dist'].ge(0) & df_tmp['relative_cell_cv_dist'].le(1/12)
]
    
choices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    
df_tmp['ring'] = np.select(conditions, choices)
df_tmp['roi_id'] = df['cell_id']

print(df_tmp.shape)
print(df_tmp.head())

(7413, 3)
   relative_cell_cv_dist  ring  roi_id
0               0.976487     1       1
1               0.976487     1       1
2               0.976487     1       1
3               0.976487     1       1
4               0.976487     1       1


In [78]:
roi_id = cell_mask.reshape(-1)
mask_df = pd.DataFrame()
mask_df['roi_id'] = roi_id
mask_df = pd.merge(mask_df, df_tmp, how = 'left', on = ['roi_id'])

print(mask_df.shape)
print(mask_df.head())

(1207298171, 3)
   roi_id  relative_cell_cv_dist  ring
0       0                    NaN     0
1       0                    NaN     0
2       0                    NaN     0
3       0                    NaN     0
4       0                    NaN     0


In [80]:
print(cell_mask.shape)
roi_id = cell_mask.reshape(-1)
print(len(roi_id))

(1452, 5208)
7562016


In [81]:
mask_df = pd.DataFrame()
mask_df['roi_id'] = roi_id

print(mask_df.shape)

(7562016, 1)


In [ ]:
mask_df2 = pd.merge(mask_df, df_tmp, how = 'left', on =['roi_id'])

print(mask_df2.shape)

In [84]:
print(df_tmp.head())

   relative_cell_cv_dist  ring  roi_id
0               0.976487     1       1
1               0.976487     1       1
2               0.976487     1       1
3               0.976487     1       1
4               0.976487     1       1


In [ ]:
mask = mask_df['ring'].to_numpy()
mask = mask.reshape(cell_mask.shape)
mask = np.uint16(mask)

### Load segmentation

###### Using existing segmentation files

<a id='alt_function'></a>

### Set paths

<a id='alt_set_path'></a>

In [ ]:
# Define file paths--------------------------------------------------------------------------

input_path = input('Input path: ')
output_path = input('Output path: ')

In [ ]:
input_path = os.path.normpath(input_path)
output_path = os.path.normpath(output_path)

### Load image

<a id='alt_load_image'></a>

In [ ]:
# Load image-----------------------------------------------------------------------------------------------
tmp_list = os.listdir(input_path)

img_list = [] 
for tmp in tmp_list:
    if re.search('.tif', tmp):
        img_list.append(tmp)
        
print(img_list)
print(len(img_list))
img_index = input('Select image: ')
img_index = int(img_index) - 1
print(img_list[img_index])

In [ ]:
img_path = os.path.join(input_path, img_list[img_index])
img = imread(img_path)

actin_ch_number = int(input('Input actin channel: ')) - 1
mito_ch_number = int(input('Input mitochondria channel: ')) - 1
lipid_ch_number = int(input('Input lipid channel: ')) - 1

In [ ]:
actin_img = img[actin_ch_number, :, :]
mito_img = img[mito_ch_number, :, :]
lipid_img = img[lipid_ch_number, :, :]

In [ ]:
cell_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cell.tif'))
cell_mask = imread(cell_mask_path)

mito_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito.tif'))
mito_mask = imread(mito_mask_path)

lipid_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid.tif'))
lipid_mask = imread(lipid_mask_path)

cv_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cv_edt.tif'))
cv_edt = imread(cv_edt_path)

pv_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_pv_edt.tif'))
pv_edt = imread(pv_edt_path)

pv_mask = np.where(pv_edt == 0, 1, 0)

mito_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito_edt.tif'))
mito_edt = imread(mito_edt_path)

lipid_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid_edt.tif'))
lipid_edt = imread(lipid_edt_path)

[Go to quantification](#quantification)

### Remake mitochondria segmentation

### Set paths

In [3]:
# Define model paths--------------------------------------------------------------------------------------

cell_model_path = input('Cell model path: ')
mito_model_path = input('Mito model path: ')
lipid_model_path = input('Lipid model path: ')

Cell model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/cell/models/brownl_cell_01
Mito model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/mito/models/brownl_mito_03
Lipid model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/lipid/models/brownl_lipid_01


In [319]:
# Define file paths--------------------------------------------------------------------------

input_path = input('Input path: ')
output_path = input('Output path: ')

Input path:  Z:\Analysis\[NCI] [TGIMB] Natalie Porat-Shliom\Lauryn Brown\images\PLIN5 variants in ND\PLIN5 (1-424)
Output path:  Z:\Analysis\[NCI] [TGIMB] Natalie Porat-Shliom\Lauryn Brown\output\PLIN5 variants in ND\PLIN5 (1-424)


In [320]:
input_path = os.path.normpath(input_path)
output_path = os.path.normpath(output_path)

cell_model_path = os.path.normpath(cell_model_path)
mito_model_path = os.path.normpath(mito_model_path)
lipid_model_path = os.path.normpath(lipid_model_path)

cell_model = models.CellposeModel(gpu = True, pretrained_model = cell_model_path)
mito_model = models.CellposeModel(gpu = True, pretrained_model = mito_model_path)
lipid_model = models.CellposeModel(gpu = True, pretrained_model = lipid_model_path)

### Load image

<a id='mito_path'></a>

In [347]:
# Load image-----------------------------------------------------------------------------------------------
tmp_list = os.listdir(input_path)

img_list = [] 
for tmp in tmp_list:
    if re.search('.tif', tmp):
        img_list.append(tmp)
        
print(img_list)
print(len(img_list))
img_index = input('Select image: ')
img_index = int(img_index) - 1
print(img_list[img_index])

['3916C M1_L1_Cropped PP-PC Axis 1.tif', '3916C M1_L1_Cropped PP-PC Axis 2.tif', '3916C M1_L2_Cropped PP-PC Axis 3.tif', '3916C M1_L2_Cropped PP-PC Axis 4.tif', '3916C M1_L2_Cropped PP-PC Axis 5.tif', '3916C M1_L3_Cropped PP-PC Axis 6.tif', '3917A M2_L1_Cropped PP-PC Axis 1.tif', '3917A M2_L1_Cropped PP-PC Axis 2.tif', '3917A M2_L2_Cropped PP-PC Axis 3.tif', '3917A M2_L2_Cropped PP-PC Axis 4.tif', '3917A M2_L3_Cropped PP-PC Axis 5.tif', '3917A M2_L3_Cropped PP-PC Axis 6.tif']
12


Select image:  2


3916C M1_L1_Cropped PP-PC Axis 2.tif


In [348]:
img_path = os.path.join(input_path, img_list[img_index])
img = imread(img_path)

print(img.shape)

viewer = napari.view_image(img)

(3, 1437, 5196)


In [349]:
actin_ch_number = int(input('Input actin channel: ')) - 1
mito_ch_number = int(input('Input mitochondria channel: ')) - 1
lipid_ch_number = int(input('Input lipid channel: ')) - 1

viewer.close()

Input actin channel:  3
Input mitochondria channel:  1
Input lipid channel:  2


In [350]:
actin_img = img[actin_ch_number, :, :]
mito_img = img[mito_ch_number, :, :]
lipid_img = img[lipid_ch_number, :, :]

In [351]:
cell_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cell.tif'))
cell_mask = imread(cell_mask_path)

#mito_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito.tif'))
#mito_mask = imread(mito_mask_path)

lipid_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid.tif'))
lipid_mask = imread(lipid_mask_path)

cv_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cv_edt.tif'))
cv_edt = imread(cv_edt_path)

pv_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_pv_edt.tif'))
pv_edt = imread(pv_edt_path)

pv_mask = np.where(pv_edt == 0, 1, 0)

#mito_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito_edt.tif'))
#mito_edt = imread(mito_edt_path)

lipid_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid_edt.tif'))
lipid_edt = imread(lipid_edt_path)

### Mitochondria segmentation

In [352]:
mito_mask = model_apply(mito_model, mito_img)
mito_mask = np.uint16(mito_mask)

In [353]:
viewer = napari.view_image(mito_img, colormap = 'green', blending = 'additive')
mito_layer = viewer.add_labels(mito_mask)

In [354]:
mito_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito.tif'))
imwrite(mito_mask_path, mito_mask)

viewer.close()

### Overlap

In [355]:
mito_binary = np.where(mito_mask > 0, 1, 0)
lipid_binary = np.where(lipid_mask > 0 , 1, 0)

overlap_tmp = np.add(mito_binary, lipid_binary)
overlap_mask = np.where(overlap_tmp == 2, cell_mask, 0)

In [356]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')
overlap_layer = viewer.add_labels(overlap_mask)
cell_layer = viewer.add_labels(cell_mask)

In [357]:
overlap_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_overlap.tif'))
imwrite(overlap_mask_path, overlap_mask)

viewer.close()

###### Mitochondria EDT map

In [358]:
mito_binary = np.where(mito_mask == 0, 1, 0)
mito_edt = nd.distance_transform_edt(mito_binary)

In [359]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')
mito_edt_layer = viewer.add_image(mito_edt)

In [360]:
mito_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito_edt.tif'))
imwrite(mito_edt_path, mito_edt)

viewer.close()

[Go to quantification](#quantification)

### Remake lipid segmentation

### Set paths

In [3]:
# Define model paths--------------------------------------------------------------------------------------

cell_model_path = input('Cell model path: ')
mito_model_path = input('Mito model path: ')
lipid_model_path = input('Lipid model path: ')

Cell model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/cell/models/brownl_cell_01
Mito model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/mito/models/brownl_mito_03
Lipid model path:  Z:/Analysis/[NCI] [TGIMB] Natalie Porat-Shliom/Lauryn Brown/training/lipid/models/brownl_lipid_01


In [656]:
# Define file paths--------------------------------------------------------------------------

input_path = input('Input path: ')
output_path = input('Output path: ')

Input path:  Z:\Analysis\[NCI] [TGIMB] Natalie Porat-Shliom\Lauryn Brown\images\PLIN5 variants in ND\PLIN5E (1-424)
Output path:  Z:\Analysis\[NCI] [TGIMB] Natalie Porat-Shliom\Lauryn Brown\output\PLIN5 variants in ND\PLIN5E (1-424)


In [657]:
input_path = os.path.normpath(input_path)
output_path = os.path.normpath(output_path)

cell_model_path = os.path.normpath(cell_model_path)
mito_model_path = os.path.normpath(mito_model_path)
lipid_model_path = os.path.normpath(lipid_model_path)

cell_model = models.CellposeModel(gpu = True, pretrained_model = cell_model_path)
mito_model = models.CellposeModel(gpu = True, pretrained_model = mito_model_path)
lipid_model = models.CellposeModel(gpu = True, pretrained_model = lipid_model_path)

### Load image

<a id='lipid_path'></a>

In [955]:
# Load image-----------------------------------------------------------------------------------------------
tmp_list = os.listdir(input_path)

img_list = [] 
for tmp in tmp_list:
    if re.search('.tif', tmp):
        img_list.append(tmp)
        
print(img_list)
print(len(img_list))
img_index = input('Select image: ')
img_index = int(img_index) - 1
print(img_list[img_index])

['3907C M1_L1_Cropped PP-PC Axis 1.tif', '3907C M1_L2_Cropped PP-PC Axis 2.tif', '3907C M1_L2_Cropped PP-PC Axis 3.tif', '3907C M1_L2_Cropped PP-PC Axis 4.tif', '3907C M1_L3_Cropped PP-PC Axis 5.tif', '3907C M1_L3_Cropped PP-PC Axis 6.tif', '3920E M2_L1_Cropped PP-Pc Axis 1.tif', '3920E M2_L1_Cropped PP-PC Axis 2.tif', '3920E M2_L1_Cropped PP-PC Axis 3.tif', '3920E M2_L1_Cropped PP-PC Axis 4.tif', '3920E M2_L2_Cropped PP-PC Axis 5.tif', '3920E M2_L2_Cropped PP-PC Axis 6.tif']
12


Select image:  12


3920E M2_L2_Cropped PP-PC Axis 6.tif


In [956]:
img_path = os.path.join(input_path, img_list[img_index])
img = imread(img_path)

print(img.shape)

viewer = napari.view_image(img)

(3, 1440, 5184)


In [957]:
actin_ch_number = int(input('Input actin channel: ')) - 1
mito_ch_number = int(input('Input mitochondria channel: ')) - 1
lipid_ch_number = int(input('Input lipid channel: ')) - 1

viewer.close()

Input actin channel:  3
Input mitochondria channel:  1
Input lipid channel:  2


In [958]:
actin_img = img[actin_ch_number, :, :]
mito_img = img[mito_ch_number, :, :]
lipid_img = img[lipid_ch_number, :, :]

In [959]:
cell_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cell.tif'))
cell_mask = imread(cell_mask_path)

mito_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito.tif'))
mito_mask = imread(mito_mask_path)

#lipid_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid.tif'))
#lipid_mask = imread(lipid_mask_path)

cv_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_cv_edt.tif'))
cv_edt = imread(cv_edt_path)

pv_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_pv_edt.tif'))
pv_edt = imread(pv_edt_path)

pv_mask = np.where(pv_edt == 0, 1, 0)

mito_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_mito_edt.tif'))
mito_edt = imread(mito_edt_path)

#lipid_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid_edt.tif'))
#lipid_edt = imread(lipid_edt_path)

### Lipid segmentation

In [960]:
lipid_mask_tmp = model_apply(lipid_model, lipid_img)
lipid_mask_tmp = np.uint16(lipid_mask_tmp)

lipid_df_tmp = object_quant(lipid_mask_tmp, lipid_img)
lipid_mask = object_filter(lipid_mask_tmp, lipid_df_tmp, 20)

100%|███████████████████████████████████████████████████████████████████████████| 1551/1551 [00:00<00:00, 41932.23it/s]


In [961]:
viewer = napari.view_image(lipid_img, colormap = 'magenta', blending = 'additive')
lipid_layer = viewer.add_labels(lipid_mask)

In [962]:
lipid_mask = viewer.layers['lipid_mask'].data

In [963]:
lipid_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid.tif'))
imwrite(lipid_mask_path, lipid_mask)

viewer.close()

### Overlap

In [964]:
mito_binary = np.where(mito_mask > 0, 1, 0)
lipid_binary = np.where(lipid_mask > 0 , 1, 0)

overlap_tmp = np.add(mito_binary, lipid_binary)
overlap_mask = np.where(overlap_tmp == 2, cell_mask, 0)

In [965]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')
overlap_layer = viewer.add_labels(overlap_mask)
cell_layer = viewer.add_labels(cell_mask)

In [966]:
overlap_mask_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_overlap.tif'))
imwrite(overlap_mask_path, overlap_mask)

viewer.close()

###### Lipid EDT map

In [967]:
lipid_binary = np.where(lipid_mask == 0, 1, 0)
lipid_edt = nd.distance_transform_edt(lipid_binary)

In [968]:
viewer = napari.view_image(actin_img, colormap = 'red', blending = 'additive')
mito_layer = viewer.add_image(mito_img, colormap = 'green', blending = 'additive')
lipid_layer = viewer.add_image(lipid_img, colormap = 'magenta', blending = 'additive')
lipid_edt_layer = viewer.add_image(lipid_edt)

In [969]:
lipid_edt_path = os.path.join(output_path, img_list[img_index].replace('.tif', '_lipid_edt.tif'))
imwrite(lipid_edt_path, lipid_edt)

viewer.close()

[Go to quantification](#quantification)